In [1]:
import numpy as np
import csv
import os
from math import sin, asin, cos, radians, fabs, sqrt
from matplotlib import pyplot as plt

In [2]:
# ROOT = '/home/rick/PythonWorkplace/CIGIT2021'
ROOT = 'E:\CIGIT2021'

In [3]:
data_name1 = 'dn2up.csv'
data_name2 = 'up2dn.csv'
label_name1 = 'dn2up_stop&signal.csv'
label_name2 = 'up2dn_stop&signal.csv'

In [4]:
data_path1 = os.path.join(ROOT, data_name1)    # dn2up
data_path2 = os.path.join(ROOT, data_name2)    # up2dn
label_path1 = os.path.join(ROOT, label_name1)  # dn2up
label_path2 = os.path.join(ROOT, label_name2)  # up2dn 

In [5]:
dn2up_data = np.loadtxt(data_path1,delimiter=',',dtype='float64')
dn2up_data = dn2up_data[:,:-10]

up2dn_data = np.loadtxt(data_path2,delimiter=',',dtype='float64')
up2dn_data = up2dn_data[:,:-10]

In [6]:
dn2up_label = np.loadtxt(label_path1,delimiter=',',dtype='int')
dn2up_label_binary = np.zeros((0,2),dtype='int')
for i in dn2up_label:
    if i[1] == 1:
        dn2up_label_binary = np.vstack((dn2up_label_binary,i))
        
dn2up_label_binary = dn2up_label_binary.T
dn2up_label = dn2up_label.T

In [7]:
up2dn_label = np.loadtxt(label_path2,delimiter=',',dtype='int')
up2dn_label_binary = np.zeros((0,2),dtype='int')
for i in up2dn_label:
    if i[1] == 1:
        up2dn_label_binary = np.vstack((up2dn_label_binary,i))
        
up2dn_label_binary = up2dn_label_binary.T
up2dn_label = up2dn_label.T

In [8]:
up2dn_label = up2dn_label[:,:-1]
up2dn_label_binary = up2dn_label_binary[:,:-1]

# Extract features

In [9]:
from scipy import stats

In [10]:
def percentile(x, percent):
    out = np.zeros(len(x[0]))
    count = 0
    y = x.T
    p = percent / 100
    for row in y:
        tmp = np.sort(row)
        location = int(len(row) * p)
        speed = tmp[location - 1]
        out[count] = speed
        count += 1
    return out

In [11]:
f1_dn2up = np.mean(dn2up_data,0)
f1_up2dn = np.mean(up2dn_data,0)

In [12]:
f2_dn2up = np.std(dn2up_data,0)
f2_up2dn = np.std(up2dn_data,0)

In [13]:
f3_dn2up = np.abs(stats.skew(dn2up_data,0))
f3_up2dn = np.abs(stats.skew(up2dn_data,0))

In [14]:
f4_dn2up = np.abs(stats.kurtosis(dn2up_data,0))
f4_up2dn = np.abs(stats.kurtosis(up2dn_data,0))

In [15]:
f5_dn2up = (f3_dn2up ** 2 + 1) / f4_dn2up
f5_up2dn = (f3_up2dn ** 2 + 1) / f4_up2dn

In [16]:
f6_dn2up = np.median(dn2up_data,0)
f6_up2dn = np.median(up2dn_data,0)

In [17]:
f7_dn2up = percentile(dn2up_data,15)
f7_up2dn = percentile(up2dn_data,15)

In [18]:
f8_dn2up = percentile(dn2up_data,85)
f8_up2dn = percentile(up2dn_data,85)

In [19]:
f9_dn2up = f8_dn2up - f7_dn2up
f9_up2dn = f8_up2dn - f7_up2dn

In [20]:
f10_dn2up = np.max(dn2up_data,0)
f10_up2dn = np.max(up2dn_data,0)

In [21]:
f11_dn2up = np.min(dn2up_data,0)
f11_up2dn = np.min(up2dn_data,0)

In [22]:
dn2up_data_feature = np.zeros((0,1890))
up2dn_data_feature = np.zeros((0,1890))

In [23]:
for i in range(1,11):
    tmp_name_dn2up = 'f%d_dn2up' %i
    tmp_name_up2dn = 'f%d_up2dn' %i
    
    dn2up_data_feature = np.vstack((dn2up_data_feature, globals()[tmp_name_dn2up]))
    up2dn_data_feature = np.vstack((up2dn_data_feature, globals()[tmp_name_up2dn]))

# Framing

In [24]:
dn2up_data_fram = np.zeros((0,len(dn2up_data),10))
up2dn_data_fram = np.zeros((0,len(up2dn_data),10))
dn2up_data_fram_feature = np.zeros((0,len(dn2up_data_feature),10))
up2dn_data_fram_feature = np.zeros((0,len(up2dn_data_feature),10))

dn2up_label_fram = np.zeros(0)
up2dn_label_fram = np.zeros(0)

In [25]:
overlap_index_dn2up = []
overlap_index_up2dn = []

In [26]:
for i in range(int(len(dn2up_data[0])/10)):    # /100m
    tmp = np.linspace(i*100,(i+1)*100-10,10,dtype=int)
    count = 0
    for j in tmp:
        
        if j in dn2up_label_binary:     # dn2up_label
            count += 1
            overlap = 5
            overlap_index_dn2up.append([i,overlap])
            for y in range((int(j/10)-7),(int(j/10)-7+overlap)):   # number of overlapping：5
                tmp_data1 = dn2up_data[:,y:y+10].reshape(1,len(dn2up_data),10)
                tmp_data2 = dn2up_data_feature[:,y:y+10].reshape(1,len(dn2up_data_feature),10)
                dn2up_data_fram = np.vstack((dn2up_data_fram,tmp_data1))
                dn2up_data_fram_feature = np.vstack((dn2up_data_fram_feature,tmp_data2))

                loc = np.argwhere(dn2up_label[0]==j)
                dn2up_label_fram = np.hstack((dn2up_label_fram,dn2up_label[1][loc[0][0]]))

        else:
            continue
            
    if count == 0:
        tmp_data1 = dn2up_data[:,i*10:i*10+10].reshape(1,len(dn2up_data),10)
        tmp_data2 = dn2up_data_feature[:,i*10:i*10+10].reshape(1,len(dn2up_data_feature),10)
        dn2up_data_fram = np.vstack((dn2up_data_fram,tmp_data1))
        dn2up_data_fram_feature = np.vstack((dn2up_data_fram_feature,tmp_data2))
        
        dn2up_label_fram = np.hstack((dn2up_label_fram,0))
    else:
        continue


In [27]:
dn2up_data_fram_feature = dn2up_data_fram_feature.reshape((len(dn2up_data_fram_feature),-1))

In [28]:
for i in range(int(len(up2dn_data[0])/10)):    # /100m
    tmp = np.linspace(i*100,(i+1)*100-10,10,dtype=int)
    count = 0
    for j in tmp:
        
        if j in up2dn_label_binary:
            count += 1
            overlap = 5
            overlap_index_up2dn.append([i,overlap])
            for y in range((int(j/10)-7),(int(j/10)-7+overlap)):   # number of overlapping：5
                tmp_data1 = up2dn_data[:,y:y+10].reshape(1,len(up2dn_data),10)
                tmp_data2 = up2dn_data_feature[:,y:y+10].reshape(1,len(up2dn_data_feature),10)
                up2dn_data_fram = np.vstack((up2dn_data_fram,tmp_data1))
                up2dn_data_fram_feature = np.vstack((up2dn_data_fram_feature,tmp_data2))

                loc = np.argwhere(up2dn_label[0]==j)
                up2dn_label_fram = np.hstack((up2dn_label_fram,up2dn_label[1][loc[0][0]]))

        else:
            continue
            
    if count == 0:
        tmp_data1 = up2dn_data[:,i*10:i*10+10].reshape(1,len(up2dn_data),10)
        tmp_data2 = up2dn_data_feature[:,i*10:i*10+10].reshape(1,len(up2dn_data_feature),10)
        up2dn_data_fram = np.vstack((up2dn_data_fram,tmp_data1))
        up2dn_data_fram_feature = np.vstack((up2dn_data_fram_feature,tmp_data2))
        
        up2dn_label_fram = np.hstack((up2dn_label_fram,0))
    else:
        continue

In [29]:
up2dn_data_fram_feature = up2dn_data_fram_feature.reshape((len(up2dn_data_fram_feature),-1))

In [30]:
data_fram_feature = np.vstack((dn2up_data_fram_feature,up2dn_data_fram_feature))
label_fram_feature = np.hstack((dn2up_label_fram,up2dn_label_fram))

# Training

# k-NN

In [31]:
from sklearn import linear_model, neighbors, metrics

In [32]:
f1_knn_dn2up = np.zeros(0)
f1_knn_up2dn = np.zeros(0)
f1_knn_all = np.zeros(0)

In [33]:
fold = 4

In [34]:
f1_knn_all_cv = np.zeros((0,fold))

In [35]:
for i in range(50):
    k_neighbors = (i+1)*2
    knn = neighbors.KNeighborsClassifier(n_neighbors=k_neighbors,algorithm="brute")
    tmp_f1_dn2up = np.zeros(0)
    tmp_f1_up2dn = np.zeros(0)
    tmp_f1_all = np.zeros(0)
    for j in range(fold):   # cross validation
        length_dn2up = int(len(dn2up_data_fram_feature) / fold)
        length_up2dn = int(len(up2dn_data_fram_feature) / fold)
        length_all = int(len(data_fram_feature) / fold)
        
        train_data_dn2up = np.vstack((dn2up_data_fram_feature[:j*length_dn2up],dn2up_data_fram_feature[(j+1)*length_dn2up:]))
        train_data_up2dn = np.vstack((up2dn_data_fram_feature[:j*length_up2dn],up2dn_data_fram_feature[(j+1)*length_up2dn:]))
        train_data_all = np.vstack((data_fram_feature[:j*length_all],data_fram_feature[(j+1)*length_all:]))

        test_data_dn2up = dn2up_data_fram_feature[j*length_dn2up:(j+1)*length_dn2up]
        test_data_up2dn = up2dn_data_fram_feature[j*length_up2dn:(j+1)*length_up2dn]
        test_data_all = data_fram_feature[j*length_all:(j+1)*length_all]
        
        train_label_dn2up = np.hstack((dn2up_label_fram[:j*length_dn2up],dn2up_label_fram[(j+1)*length_dn2up:]))
        test_label_dn2up = dn2up_label_fram[j*length_dn2up:(j+1)*length_dn2up]
        
        train_label_up2dn = np.hstack((up2dn_label_fram[:j*length_up2dn],up2dn_label_fram[(j+1)*length_up2dn:]))
        test_label_up2dn = up2dn_label_fram[j*length_up2dn:(j+1)*length_up2dn]
        
        train_label_all = np.hstack((label_fram_feature[:j*length_all],label_fram_feature[(j+1)*length_all:]))
        test_label_all = label_fram_feature[j*length_all:(j+1)*length_all]
        
        knn.fit(train_data_dn2up,train_label_dn2up) 
        knn_pred_dn2up = knn.predict(test_data_dn2up)        
        f1score_dn2up = metrics.f1_score(knn_pred_dn2up,test_label_dn2up)
        tmp_f1_dn2up = np.hstack((tmp_f1_dn2up,f1score_dn2up))
        
        knn.fit(train_data_up2dn,train_label_up2dn) 
        knn_pred_up2dn = knn.predict(test_data_up2dn)        
        f1score_up2dn = metrics.f1_score(knn_pred_up2dn,test_label_up2dn)
        tmp_f1_up2dn = np.hstack((tmp_f1_up2dn,f1score_up2dn))
        
        knn.fit(train_data_all,train_label_all) 
        knn_pred_all = knn.predict(test_data_all)        
        f1score_all = metrics.f1_score(knn_pred_all,test_label_all)
        tmp_f1_all = np.hstack((tmp_f1_all,f1score_all))
        
    f1_m_dn2up = np.max(tmp_f1_dn2up)
    f1_m_up2dn = np.max(tmp_f1_up2dn)
    f1_m_all = np.max(tmp_f1_all)
    f1_knn_all_cv = np.vstack((f1_knn_all_cv,tmp_f1_all))
    
    f1_knn_dn2up = np.hstack((f1_knn_dn2up,f1_m_dn2up))
    f1_knn_up2dn = np.hstack((f1_knn_up2dn,f1_m_up2dn))
    f1_knn_all = np.hstack((f1_knn_all,f1_m_all))
    
f1_max_dn2up_knn = np.max(f1_knn_dn2up)
f1_max_up2dn_knn = np.max(f1_knn_up2dn)
f1_max_all_knn = np.max(f1_knn_all)

In [36]:
index1 = np.where(f1_knn_all == f1_max_all_knn)[0][0]
index2 = np.where(f1_knn_all_cv[index1] == f1_max_all_knn)[0][0]

In [37]:
# verify the highest f1
result_knn_data_train = np.vstack((data_fram_feature[:index2*length_all],data_fram_feature[(index2+1)*length_all:]))
result_knn_data_test = data_fram_feature[index2*length_all:(index2+1)*length_all]
result_knn_label_train = np.hstack((label_fram_feature[:index2*length_all],label_fram_feature[(index2+1)*length_all:]))
result_knn_label_test = label_fram_feature[index2*length_all:(index2+1)*length_all]

result_k_neighbors = (index1+1)*2
result_knn = neighbors.KNeighborsClassifier(n_neighbors=result_k_neighbors,algorithm="brute")

result_knn.fit(result_knn_data_train,result_knn_label_train) 
result_knn_pred = result_knn.predict(result_knn_data_test)
f1_verify_knn = metrics.f1_score(result_knn_pred,result_knn_label_test)

In [41]:
print('max f1 score for all:',f1_max_all_knn,'\n' 'max f1 score for up2dn:',f1_max_up2dn_knn,'\n' 'max f1 score for dn2up:',f1_max_dn2up_knn)

max f1 score for all: 0.7567567567567567 
max f1 score for up2dn: 0.8648648648648648 
max f1 score for dn2up: 0.8035714285714286


# SVM

In [42]:
from sklearn import svm

In [43]:
f1_svm_dn2up = np.zeros(0)
f1_svm_up2dn = np.zeros(0)
f1_svm_all = np.zeros(0)

In [44]:
f1_svm_all_cv = np.zeros((0,fold))

In [45]:
for i in range(-4,10):
    c = pow(2,i)
    g = 1
    SVM = svm.SVC(C=c,kernel='rbf',gamma=g)
    tmp_f1_dn2up = np.zeros(0)
    tmp_f1_up2dn = np.zeros(0)
    tmp_f1_all = np.zeros(0)
    for j in range(fold):   # cross validation
        length_dn2up = int(len(dn2up_data_fram_feature) / fold)
        length_up2dn = int(len(up2dn_data_fram_feature) / fold)
        length_all = int(len(data_fram_feature) / fold)
        
        train_data_dn2up = np.vstack((dn2up_data_fram_feature[:j*length_dn2up],dn2up_data_fram_feature[(j+1)*length_dn2up:]))
        train_data_up2dn = np.vstack((up2dn_data_fram_feature[:j*length_up2dn],up2dn_data_fram_feature[(j+1)*length_up2dn:]))
        train_data_all = np.vstack((data_fram_feature[:j*length_all],data_fram_feature[(j+1)*length_all:]))

        test_data_dn2up = dn2up_data_fram_feature[j*length_dn2up:(j+1)*length_dn2up]
        test_data_up2dn = up2dn_data_fram_feature[j*length_up2dn:(j+1)*length_up2dn]
        test_data_all = data_fram_feature[j*length_all:(j+1)*length_all]
        
        train_label_dn2up = np.hstack((dn2up_label_fram[:j*length_dn2up],dn2up_label_fram[(j+1)*length_dn2up:]))
        test_label_dn2up = dn2up_label_fram[j*length_dn2up:(j+1)*length_dn2up]
        
        train_label_up2dn = np.hstack((up2dn_label_fram[:j*length_up2dn],up2dn_label_fram[(j+1)*length_up2dn:]))
        test_label_up2dn = up2dn_label_fram[j*length_up2dn:(j+1)*length_up2dn]
        
        train_label_all = np.hstack((label_fram_feature[:j*length_all],label_fram_feature[(j+1)*length_all:]))
        test_label_all = label_fram_feature[j*length_all:(j+1)*length_all]
        
        SVM.fit(train_data_dn2up,train_label_dn2up) 
        svm_pred_dn2up = SVM.predict(test_data_dn2up)        
        f1score_dn2up = metrics.f1_score(svm_pred_dn2up,test_label_dn2up)
        tmp_f1_dn2up = np.hstack((tmp_f1_dn2up,f1score_dn2up))
        
        SVM.fit(train_data_up2dn,train_label_up2dn) 
        svm_pred_up2dn = SVM.predict(test_data_up2dn)        
        f1score_up2dn = metrics.f1_score(svm_pred_up2dn,test_label_up2dn)
        tmp_f1_up2dn = np.hstack((tmp_f1_up2dn,f1score_up2dn))
        
        SVM.fit(train_data_all,train_label_all) 
        svm_pred_all = SVM.predict(test_data_all)        
        f1score_all = metrics.f1_score(svm_pred_all,test_label_all)
        tmp_f1_all = np.hstack((tmp_f1_all,f1score_all))
        
    f1_m_dn2up = np.max(tmp_f1_dn2up)
    f1_m_up2dn = np.max(tmp_f1_up2dn)
    f1_m_all = np.max(tmp_f1_all)
    f1_svm_all_cv = np.vstack((f1_svm_all_cv,tmp_f1_all))
    
    f1_svm_dn2up = np.hstack((f1_svm_dn2up,f1_m_dn2up))
    f1_svm_up2dn = np.hstack((f1_svm_up2dn,f1_m_up2dn))
    f1_svm_all = np.hstack((f1_svm_all,f1_m_all))
    
f1_max_dn2up_svm = np.max(f1_svm_dn2up)
f1_max_up2dn_svm = np.max(f1_svm_up2dn)
f1_max_all_svm = np.max(f1_svm_all)

In [46]:
index3 = np.where(f1_svm_all == f1_max_all_svm)[0][0]
index4 = np.where(f1_svm_all_cv[index3] == f1_max_all_svm)[0][0]

In [47]:
# verify the highest f1
result_svm_data_train = np.vstack((data_fram_feature[:index4*length_all],data_fram_feature[(index4+1)*length_all:]))
result_svm_data_test = data_fram_feature[index4*length_all:(index4+1)*length_all]
result_svm_label_train = np.hstack((label_fram_feature[:index4*length_all],label_fram_feature[(index4+1)*length_all:]))
result_svm_label_test = label_fram_feature[index4*length_all:(index4+1)*length_all]

c = pow(2, int(index3-4))
result_svm = svm.SVC(C=c,kernel='rbf',gamma=g)

result_svm.fit(result_svm_data_train,result_svm_label_train) 
result_svm_pred = result_svm.predict(result_svm_data_test)
f1_verify_svm = metrics.f1_score(result_svm_pred,result_svm_label_test)

In [48]:
print('max f1 score for all:',f1_max_all_svm,'\n' 'max f1 score for up2dn:',f1_max_up2dn_svm,'\n' 'max f1 score for dn2up:',f1_max_dn2up_svm)

max f1 score for all: 0.7086614173228347 
max f1 score for up2dn: 0.7669172932330827 
max f1 score for dn2up: 0.7575757575757575


# RF

In [49]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
f1_rf_dn2up = np.zeros(0)
f1_rf_up2dn = np.zeros(0)
f1_rf_all = np.zeros(0)

In [51]:
f1_rf_all_cv = np.zeros((0,fold))

In [52]:
for i in range(10):
    n = (i+1)*10
    RF = RandomForestClassifier(n_estimators=n, criterion='entropy')
    tmp_f1_dn2up = np.zeros(0)
    tmp_f1_up2dn = np.zeros(0)
    tmp_f1_all = np.zeros(0)
    for j in range(fold):   # cross validation
        length_dn2up = int(len(dn2up_data_fram_feature) / fold)
        length_up2dn = int(len(up2dn_data_fram_feature) / fold)
        length_all = int(len(data_fram_feature) / fold)
        
        train_data_dn2up = np.vstack((dn2up_data_fram_feature[:j*length_dn2up],dn2up_data_fram_feature[(j+1)*length_dn2up:]))
        train_data_up2dn = np.vstack((up2dn_data_fram_feature[:j*length_up2dn],up2dn_data_fram_feature[(j+1)*length_up2dn:]))
        train_data_all = np.vstack((data_fram_feature[:j*length_all],data_fram_feature[(j+1)*length_all:]))

        test_data_dn2up = dn2up_data_fram_feature[j*length_dn2up:(j+1)*length_dn2up]
        test_data_up2dn = up2dn_data_fram_feature[j*length_up2dn:(j+1)*length_up2dn]
        test_data_all = data_fram_feature[j*length_all:(j+1)*length_all]
        
        train_label_dn2up = np.hstack((dn2up_label_fram[:j*length_dn2up],dn2up_label_fram[(j+1)*length_dn2up:]))
        test_label_dn2up = dn2up_label_fram[j*length_dn2up:(j+1)*length_dn2up]
        
        train_label_up2dn = np.hstack((up2dn_label_fram[:j*length_up2dn],up2dn_label_fram[(j+1)*length_up2dn:]))
        test_label_up2dn = up2dn_label_fram[j*length_up2dn:(j+1)*length_up2dn]
        
        train_label_all = np.hstack((label_fram_feature[:j*length_all],label_fram_feature[(j+1)*length_all:]))
        test_label_all = label_fram_feature[j*length_all:(j+1)*length_all]
        
        RF.fit(train_data_dn2up,train_label_dn2up) 
        rf_pred_dn2up = RF.predict(test_data_dn2up)        
        f1score_dn2up = metrics.f1_score(rf_pred_dn2up,test_label_dn2up)
        tmp_f1_dn2up = np.hstack((tmp_f1_dn2up,f1score_dn2up))
        
        RF.fit(train_data_up2dn,train_label_up2dn) 
        rf_pred_up2dn = RF.predict(test_data_up2dn)        
        f1score_up2dn = metrics.f1_score(rf_pred_up2dn,test_label_up2dn)
        tmp_f1_up2dn = np.hstack((tmp_f1_up2dn,f1score_up2dn))
        
        RF.fit(train_data_all,train_label_all) 
        rf_pred_all = RF.predict(test_data_all)        
        f1score_all = metrics.f1_score(rf_pred_all,test_label_all)
        tmp_f1_all = np.hstack((tmp_f1_all,f1score_all))
        
    f1_m_dn2up = np.max(tmp_f1_dn2up)
    f1_m_up2dn = np.max(tmp_f1_up2dn)
    f1_m_all = np.max(tmp_f1_all)
    f1_rf_all_cv = np.vstack((f1_rf_all_cv,tmp_f1_all))
    
    f1_rf_dn2up = np.hstack((f1_rf_dn2up,f1_m_dn2up))
    f1_rf_up2dn = np.hstack((f1_rf_up2dn,f1_m_up2dn))
    f1_rf_all = np.hstack((f1_rf_all,f1_m_all))
    
f1_max_dn2up_rf = np.max(f1_rf_dn2up)
f1_max_up2dn_rf = np.max(f1_rf_up2dn)
f1_max_all_rf = np.max(f1_rf_all)

In [53]:
print('max f1 score for all:',f1_max_all_rf,'\n' 'max f1 score for up2dn:',f1_max_up2dn_rf,'\n' 'max f1 score for dn2up:',f1_max_dn2up_rf)

max f1 score for all: 0.654320987654321 
max f1 score for up2dn: 0.6153846153846154 
max f1 score for dn2up: 0.7532467532467533


# MLP

In [31]:
import torch
from torch import nn, optim
from visdom import Visdom
import Ipynb_importer

In [32]:
from fc import FullConnect

importing Jupyter notebook from fc.ipynb


In [43]:
batchsz = 32
device = torch.device('cuda')
model = FullConnect()
criteon = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

In [60]:
viz = Visdom()
viz.line([0.], [0.], win='Training_Loss', opts=dict(title='Training loss'))
viz.line([0.], [0.], win='Training_Accuracy', opts=dict(title='Training accuracy'))
viz.line([0.], [0.], win='Testing_Loss', opts=dict(title='Testing loss'))
viz.line([0.], [0.], win='Accuracy', opts=dict(title='Accuracy'))

Setting up a new session...


'Accuracy'

In [46]:
data = data_fram_feature
label = label_fram_feature

In [47]:
# 随机打乱
np.random.seed(10)
np.random.shuffle(data)
np.random.shuffle(label)

In [48]:
train_data = torch.from_numpy(data[int(len(data)/3):])
train_data = train_data.type(torch.FloatTensor)

test_data = torch.from_numpy(data[:int(len(data)/3)])
test_data = test_data.type(torch.FloatTensor)

train_label = torch.from_numpy(label[int(len(data)/3):])
train_label = torch.as_tensor(train_label, dtype=torch.long)

test_label = torch.from_numpy(label[:int(len(data)/3)])
test_label = torch.as_tensor(test_label, dtype=torch.long)

In [61]:
global_step = 0
for epoch in range(200):
    model.train()
    for i in range(int(len(train_data) / batchsz)):
        x = train_data[batchsz * i: batchsz * (i + 1)]
        label = train_label[batchsz * i: batchsz * (i + 1)]
        x, label = x.to(device), label.to(device)
        
        logits = model(x).squeeze(1)
        loss = criteon(logits, label)
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='Training_Loss', update='append')

        print('epoch:', epoch, 'loss:', loss.item())
    
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_num = 0
        test_loss = 0

        for i in range(int(len(test_data) / batchsz)):  
            
            # label: [100]
            x = test_data[batchsz * i: batchsz * (i + 1)]
            label = test_label[batchsz * i: batchsz * (i + 1)]
            x, label = x.to(device), label.to(device)
            
            # logits: [100, 2]
            logits = model(x).squeeze(1)
            test_loss += criteon(logits, label).item()

            pred = logits.argmax(dim=1)
            total_correct += torch.eq(pred, label).float().sum().item()
            total_num += x.size(0)

        acc = total_correct / total_num
        print('epoch:', epoch, 'acc:', acc)
        # print('logits:', logits)

        viz.line([test_loss], [global_step], win='Testing_Loss', opts=dict(title='Testing loss'), update='append')
        viz.line([acc], [global_step], win='Accuracy', opts=dict(title='Accuracy'), update='append')

epoch: 0 loss: 0.3861740827560425
epoch: 0 loss: 0.324104368686676
epoch: 0 loss: 0.31272953748703003
epoch: 0 loss: 0.40726104378700256
epoch: 0 loss: 0.35409513115882874
epoch: 0 loss: 0.44465216994285583
epoch: 0 loss: 0.3778573274612427
epoch: 0 loss: 0.2814750075340271
epoch: 0 loss: 0.4937072992324829
epoch: 0 loss: 0.35421937704086304
epoch: 0 loss: 0.25649887323379517
epoch: 0 loss: 0.35524553060531616
epoch: 0 loss: 0.3686962127685547
epoch: 0 acc: 0.5520833333333334
epoch: 1 loss: 0.4094166159629822
epoch: 1 loss: 0.30124980211257935
epoch: 1 loss: 0.29859083890914917
epoch: 1 loss: 0.43501898646354675
epoch: 1 loss: 0.33196181058883667
epoch: 1 loss: 0.4559423625469208
epoch: 1 loss: 0.340625137090683
epoch: 1 loss: 0.24398775398731232
epoch: 1 loss: 0.5003544092178345
epoch: 1 loss: 0.34835654497146606
epoch: 1 loss: 0.27132871747016907
epoch: 1 loss: 0.3194340765476227
epoch: 1 loss: 0.3573673665523529
epoch: 1 acc: 0.5520833333333334
epoch: 2 loss: 0.377846896648407
epoch

epoch: 18 loss: 0.28071513772010803
epoch: 18 loss: 0.46539410948753357
epoch: 18 loss: 0.3628064692020416
epoch: 18 loss: 0.2294388711452484
epoch: 18 loss: 0.3308197259902954
epoch: 18 loss: 0.3566410541534424
epoch: 18 acc: 0.5572916666666666
epoch: 19 loss: 0.4729793965816498
epoch: 19 loss: 0.29628875851631165
epoch: 19 loss: 0.32609784603118896
epoch: 19 loss: 0.3559989929199219
epoch: 19 loss: 0.33615678548812866
epoch: 19 loss: 0.4541664719581604
epoch: 19 loss: 0.3858971893787384
epoch: 19 loss: 0.2799304127693176
epoch: 19 loss: 0.48633530735969543
epoch: 19 loss: 0.3638972043991089
epoch: 19 loss: 0.24187451601028442
epoch: 19 loss: 0.3577767014503479
epoch: 19 loss: 0.37023085355758667
epoch: 19 acc: 0.5416666666666666
epoch: 20 loss: 0.4762517511844635
epoch: 20 loss: 0.3104972839355469
epoch: 20 loss: 0.32735490798950195
epoch: 20 loss: 0.38776859641075134
epoch: 20 loss: 0.3334616422653198
epoch: 20 loss: 0.4182613790035248
epoch: 20 loss: 0.4278113543987274
epoch: 20 lo

epoch: 35 loss: 0.3348240554332733
epoch: 35 loss: 0.38881900906562805
epoch: 35 loss: 0.35052987933158875
epoch: 35 loss: 0.2556016743183136
epoch: 35 loss: 0.4604160785675049
epoch: 35 loss: 0.33103829622268677
epoch: 35 loss: 0.20690332353115082
epoch: 35 loss: 0.3214397728443146
epoch: 35 loss: 0.3081083297729492
epoch: 35 acc: 0.546875
epoch: 36 loss: 0.4079740047454834
epoch: 36 loss: 0.2774033844470978
epoch: 36 loss: 0.2920841872692108
epoch: 36 loss: 0.37752455472946167
epoch: 36 loss: 0.3447597324848175
epoch: 36 loss: 0.4061150848865509
epoch: 36 loss: 0.32931870222091675
epoch: 36 loss: 0.23712459206581116
epoch: 36 loss: 0.45887264609336853
epoch: 36 loss: 0.3270438015460968
epoch: 36 loss: 0.21405398845672607
epoch: 36 loss: 0.31291255354881287
epoch: 36 loss: 0.30296194553375244
epoch: 36 acc: 0.5416666666666666
epoch: 37 loss: 0.39949604868888855
epoch: 37 loss: 0.28319621086120605
epoch: 37 loss: 0.30082839727401733
epoch: 37 loss: 0.36711978912353516
epoch: 37 loss: 0

epoch: 52 loss: 0.2842666506767273
epoch: 52 loss: 0.3404858112335205
epoch: 52 loss: 0.3261762261390686
epoch: 52 loss: 0.36585715413093567
epoch: 52 loss: 0.29812178015708923
epoch: 52 loss: 0.21981163322925568
epoch: 52 loss: 0.46460163593292236
epoch: 52 loss: 0.3308047950267792
epoch: 52 loss: 0.2161634862422943
epoch: 52 loss: 0.3041692078113556
epoch: 52 loss: 0.2954905927181244
epoch: 52 acc: 0.546875
epoch: 53 loss: 0.33713972568511963
epoch: 53 loss: 0.28063932061195374
epoch: 53 loss: 0.2838468551635742
epoch: 53 loss: 0.34798476099967957
epoch: 53 loss: 0.3276677429676056
epoch: 53 loss: 0.3691447377204895
epoch: 53 loss: 0.30532845854759216
epoch: 53 loss: 0.22020266950130463
epoch: 53 loss: 0.45500612258911133
epoch: 53 loss: 0.34198829531669617
epoch: 53 loss: 0.20782962441444397
epoch: 53 loss: 0.3027105927467346
epoch: 53 loss: 0.30488717555999756
epoch: 53 acc: 0.5572916666666666
epoch: 54 loss: 0.34933555126190186
epoch: 54 loss: 0.280878484249115
epoch: 54 loss: 0.2

epoch: 70 loss: 0.31276506185531616
epoch: 70 loss: 0.28905051946640015
epoch: 70 loss: 0.284274697303772
epoch: 70 loss: 0.3273395299911499
epoch: 70 loss: 0.32261723279953003
epoch: 70 loss: 0.35834577679634094
epoch: 70 loss: 0.2924129068851471
epoch: 70 loss: 0.22260917723178864
epoch: 70 loss: 0.4609816074371338
epoch: 70 loss: 0.34583449363708496
epoch: 70 loss: 0.2110205739736557
epoch: 70 loss: 0.2859140932559967
epoch: 70 loss: 0.2954082489013672
epoch: 70 acc: 0.5416666666666666
epoch: 71 loss: 0.320983350276947
epoch: 71 loss: 0.2792871594429016
epoch: 71 loss: 0.28169694542884827
epoch: 71 loss: 0.3424092233181
epoch: 71 loss: 0.3147197365760803
epoch: 71 loss: 0.3414901793003082
epoch: 71 loss: 0.2862316370010376
epoch: 71 loss: 0.21666216850280762
epoch: 71 loss: 0.4655066132545471
epoch: 71 loss: 0.3466825783252716
epoch: 71 loss: 0.2142067402601242
epoch: 71 loss: 0.28912246227264404
epoch: 71 loss: 0.2896598279476166
epoch: 71 acc: 0.5625
epoch: 72 loss: 0.311680674552

...

epoch: 120 acc: 0.53125
epoch: 121 loss: 0.29732707142829895
epoch: 121 loss: 0.28835225105285645
epoch: 121 loss: 0.2841212749481201
epoch: 121 loss: 0.3027336597442627
epoch: 121 loss: 0.312640517950058
epoch: 121 loss: 0.3055502772331238
epoch: 121 loss: 0.283718079328537
epoch: 121 loss: 0.20909562706947327
epoch: 121 loss: 0.4539532959461212
epoch: 121 loss: 0.3693595230579376
epoch: 121 loss: 0.1954742670059204
epoch: 121 loss: 0.2773561179637909
epoch: 121 loss: 0.28534433245658875
epoch: 121 acc: 0.5520833333333334
epoch: 122 loss: 0.2914878726005554
epoch: 122 loss: 0.3099711239337921
epoch: 122 loss: 0.2940388023853302
epoch: 122 loss: 0.318199098110199
epoch: 122 loss: 0.2993839681148529
epoch: 122 loss: 0.31457701325416565
epoch: 122 loss: 0.2785241901874542
epoch: 122 loss: 0.2044382095336914
epoch: 122 loss: 0.454499214887619
epoch: 122 loss: 0.3735322058200836
epoch: 122 loss: 0.18667834997177124
epoch: 122 loss: 0.2674484848976135
epoch: 122 loss: 0.28351297974586487
ep

epoch: 137 loss: 0.21632355451583862
epoch: 137 loss: 0.45033344626426697
epoch: 137 loss: 0.3787553310394287
epoch: 137 loss: 0.1843523383140564
epoch: 137 loss: 0.25810515880584717
epoch: 137 loss: 0.2767687141895294
epoch: 137 acc: 0.5416666666666666
epoch: 138 loss: 0.2770662307739258
epoch: 138 loss: 0.3252318799495697
epoch: 138 loss: 0.3089018166065216
epoch: 138 loss: 0.31123778223991394
epoch: 138 loss: 0.28003743290901184
epoch: 138 loss: 0.2997485101222992
epoch: 138 loss: 0.2614370286464691
epoch: 138 loss: 0.19879460334777832
epoch: 138 loss: 0.44927555322647095
epoch: 138 loss: 0.38276034593582153
epoch: 138 loss: 0.17987555265426636
epoch: 138 loss: 0.2828817069530487
epoch: 138 loss: 0.2807132601737976
epoch: 138 acc: 0.515625
epoch: 139 loss: 0.28844910860061646
epoch: 139 loss: 0.27634549140930176
epoch: 139 loss: 0.278911828994751
epoch: 139 loss: 0.3001590371131897
epoch: 139 loss: 0.28190404176712036
epoch: 139 loss: 0.3281398117542267
epoch: 139 loss: 0.2521929740

epoch: 154 loss: 0.3353573679924011
epoch: 154 loss: 0.38677722215652466
epoch: 154 loss: 0.3033275902271271
epoch: 154 loss: 0.4641043543815613
epoch: 154 loss: 0.2912125587463379
epoch: 154 loss: 0.4366849362850189
epoch: 154 loss: 0.3338659405708313
epoch: 154 loss: 0.21299007534980774
epoch: 154 loss: 0.5126045346260071
epoch: 154 loss: 0.38142386078834534
epoch: 154 loss: 0.23518258333206177
epoch: 154 loss: 0.2907477915287018
epoch: 154 loss: 0.2883723974227905
epoch: 154 acc: 0.5416666666666666
epoch: 155 loss: 0.32837533950805664
epoch: 155 loss: 0.34971359372138977
epoch: 155 loss: 0.32860705256462097
epoch: 155 loss: 0.4162665605545044
epoch: 155 loss: 0.31588152050971985
epoch: 155 loss: 0.47037553787231445
epoch: 155 loss: 0.26349446177482605
epoch: 155 loss: 0.1990281194448471
epoch: 155 loss: 0.46855056285858154
epoch: 155 loss: 0.43751227855682373
epoch: 155 loss: 0.19931668043136597
epoch: 155 loss: 0.3064453601837158
epoch: 155 loss: 0.2765137851238251
epoch: 155 acc: 

epoch: 170 loss: 0.23362389206886292
epoch: 170 loss: 0.4702720642089844
epoch: 170 loss: 0.3544173538684845
epoch: 170 loss: 0.2518898844718933
epoch: 170 loss: 0.29973074793815613
epoch: 170 loss: 0.3348163068294525
epoch: 170 acc: 0.5364583333333334
epoch: 171 loss: 0.31773507595062256
epoch: 171 loss: 0.32132112979888916
epoch: 171 loss: 0.3234366178512573
epoch: 171 loss: 0.4633745849132538
epoch: 171 loss: 0.35792601108551025
epoch: 171 loss: 0.4781886637210846
epoch: 171 loss: 0.24897518754005432
epoch: 171 loss: 0.21238315105438232
epoch: 171 loss: 0.47030866146087646
epoch: 171 loss: 0.367419958114624
epoch: 171 loss: 0.24631398916244507
epoch: 171 loss: 0.2910158634185791
epoch: 171 loss: 0.35681119561195374
epoch: 171 acc: 0.5364583333333334
epoch: 172 loss: 0.3047420382499695
epoch: 172 loss: 0.34231850504875183
epoch: 172 loss: 0.3351570665836334
epoch: 172 loss: 0.5178828239440918
epoch: 172 loss: 0.35111507773399353
epoch: 172 loss: 0.43967586755752563
epoch: 172 loss: 0

epoch: 187 loss: 0.41202086210250854
epoch: 187 loss: 0.32374998927116394
epoch: 187 loss: 0.573282778263092
epoch: 187 loss: 0.40223294496536255
epoch: 187 loss: 0.28316766023635864
epoch: 187 loss: 0.4120195806026459
epoch: 187 loss: 0.39284563064575195
epoch: 187 acc: 0.5520833333333334
epoch: 188 loss: 0.5179316401481628
epoch: 188 loss: 0.4339301586151123
epoch: 188 loss: 0.3813290297985077
epoch: 188 loss: 0.41175177693367004
epoch: 188 loss: 0.42741307616233826
epoch: 188 loss: 0.5142290592193604
epoch: 188 loss: 0.4281335473060608
epoch: 188 loss: 0.3567453920841217
epoch: 188 loss: 0.5242341160774231
epoch: 188 loss: 0.3445911705493927
epoch: 188 loss: 0.24136851727962494
epoch: 188 loss: 0.36213478446006775
epoch: 188 loss: 0.3854679763317108
epoch: 188 acc: 0.5677083333333334
epoch: 189 loss: 0.48446783423423767
epoch: 189 loss: 0.3826647400856018
epoch: 189 loss: 0.38720810413360596
epoch: 189 loss: 0.5445411801338196
epoch: 189 loss: 0.3297213613986969
epoch: 189 loss: 0.4

# CNN

In [62]:
from lenet import Lenet5

In [65]:
data = np.expand_dims(up2dn_data_fram,axis=1)
label = up2dn_label_fram

In [67]:
# 随机打乱
np.random.seed(5)
np.random.shuffle(data)
np.random.shuffle(label)

In [68]:
train_data = torch.from_numpy(data[int(len(data)/3):])
train_data = train_data.type(torch.FloatTensor)

test_data = torch.from_numpy(data[:int(len(data)/3)])
test_data = test_data.type(torch.FloatTensor)

train_label = torch.from_numpy(label[int(len(data)/3):])
train_label = torch.as_tensor(train_label, dtype=torch.long)

test_label = torch.from_numpy(label[:int(len(data)/3)])
test_label = torch.as_tensor(test_label, dtype=torch.long)

In [69]:
model = Lenet5().to(device)

In [70]:
viz = Visdom()
viz.line([0.], [0.], win='Training_Loss', opts=dict(title='Training loss'))
viz.line([0.], [0.], win='Training_Accuracy', opts=dict(title='Training accuracy'))
viz.line([0.], [0.], win='Testing_Loss', opts=dict(title='Testing loss'))
viz.line([0.], [0.], win='Accuracy', opts=dict(title='Accuracy'))

Setting up a new session...


'Accuracy'

In [71]:
global_step = 0
for epoch in range(100):
    model.train()
    for i in range(int(len(train_data) / batchsz)):
        x = train_data[batchsz * i: batchsz * (i + 1)]
        label = train_label[batchsz * i: batchsz * (i + 1)]
        x, label = x.to(device), label.to(device)
        
        logits = model(x).squeeze(1)
        loss = criteon(logits, label)
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='Training_Loss', update='append')

        print('epoch:', epoch, 'loss:', loss.item())
    
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_num = 0
        test_loss = 0

        for i in range(int(len(test_data) / batchsz)):  
            
            x = test_data[batchsz * i: batchsz * (i + 1)]
            label = test_label[batchsz * i: batchsz * (i + 1)]
            x, label = x.to(device), label.to(device)
            
            logits = model(x).squeeze(1)
            test_loss += criteon(logits, label).item()

            pred = logits.argmax(dim=1)
            total_correct += torch.eq(pred, label).float().sum().item()
            total_num += x.size(0)

        acc = total_correct / total_num
        print('epoch:', epoch, 'acc:', acc)
        # print('logits:', logits)

        viz.line([test_loss], [global_step], win='Testing_Loss', opts=dict(title='Testing loss'), update='append')
        viz.line([acc], [global_step], win='Accuracy', opts=dict(title='Accuracy'), update='append')

epoch: 0 loss: 0.6988680958747864
epoch: 0 loss: 0.6938843131065369
epoch: 0 loss: 0.697181761264801
epoch: 0 loss: 0.6860491037368774
epoch: 0 loss: 0.7120947241783142
epoch: 0 loss: 0.7033352851867676
epoch: 0 acc: 0.5208333333333334
epoch: 1 loss: 0.6988680958747864
epoch: 1 loss: 0.6938843131065369
epoch: 1 loss: 0.697181761264801
epoch: 1 loss: 0.6860491037368774
epoch: 1 loss: 0.7120947241783142
epoch: 1 loss: 0.7033352851867676
epoch: 1 acc: 0.5208333333333334
epoch: 2 loss: 0.6988680958747864
epoch: 2 loss: 0.6938843131065369
epoch: 2 loss: 0.697181761264801
epoch: 2 loss: 0.6860491037368774
epoch: 2 loss: 0.7120947241783142
epoch: 2 loss: 0.7033352851867676
epoch: 2 acc: 0.5208333333333334
epoch: 3 loss: 0.6988680958747864
epoch: 3 loss: 0.6938843131065369
epoch: 3 loss: 0.697181761264801
epoch: 3 loss: 0.6860491037368774
epoch: 3 loss: 0.7120947241783142
epoch: 3 loss: 0.7033352851867676
epoch: 3 acc: 0.5208333333333334
epoch: 4 loss: 0.6988680958747864
epoch: 4 loss: 0.69388

epoch: 35 loss: 0.6938843131065369
epoch: 35 loss: 0.697181761264801
epoch: 35 loss: 0.6860491037368774
epoch: 35 loss: 0.7120947241783142
epoch: 35 loss: 0.7033352851867676
epoch: 35 acc: 0.5208333333333334
epoch: 36 loss: 0.6988680958747864
epoch: 36 loss: 0.6938843131065369
epoch: 36 loss: 0.697181761264801
epoch: 36 loss: 0.6860491037368774
epoch: 36 loss: 0.7120947241783142
epoch: 36 loss: 0.7033352851867676
epoch: 36 acc: 0.5208333333333334
epoch: 37 loss: 0.6988680958747864
epoch: 37 loss: 0.6938843131065369
epoch: 37 loss: 0.697181761264801
epoch: 37 loss: 0.6860491037368774
epoch: 37 loss: 0.7120947241783142
epoch: 37 loss: 0.7033352851867676
epoch: 37 acc: 0.5208333333333334
epoch: 38 loss: 0.6988680958747864
epoch: 38 loss: 0.6938843131065369
epoch: 38 loss: 0.697181761264801
epoch: 38 loss: 0.6860491037368774
epoch: 38 loss: 0.7120947241783142
epoch: 38 loss: 0.7033352851867676
epoch: 38 acc: 0.5208333333333334
epoch: 39 loss: 0.6988680958747864
epoch: 39 loss: 0.6938843131

epoch: 70 loss: 0.7033352851867676
epoch: 70 acc: 0.5208333333333334
epoch: 71 loss: 0.6988680958747864
epoch: 71 loss: 0.6938843131065369
epoch: 71 loss: 0.697181761264801
epoch: 71 loss: 0.6860491037368774
epoch: 71 loss: 0.7120947241783142
epoch: 71 loss: 0.7033352851867676
epoch: 71 acc: 0.5208333333333334
epoch: 72 loss: 0.6988680958747864
epoch: 72 loss: 0.6938843131065369
epoch: 72 loss: 0.697181761264801
epoch: 72 loss: 0.6860491037368774
epoch: 72 loss: 0.7120947241783142
epoch: 72 loss: 0.7033352851867676
epoch: 72 acc: 0.5208333333333334
epoch: 73 loss: 0.6988680958747864
epoch: 73 loss: 0.6938843131065369
epoch: 73 loss: 0.697181761264801
epoch: 73 loss: 0.6860491037368774
epoch: 73 loss: 0.7120947241783142
epoch: 73 loss: 0.7033352851867676
epoch: 73 acc: 0.5208333333333334
epoch: 74 loss: 0.6988680958747864
epoch: 74 loss: 0.6938843131065369
epoch: 74 loss: 0.697181761264801
epoch: 74 loss: 0.6860491037368774
epoch: 74 loss: 0.7120947241783142
epoch: 74 loss: 0.7033352851